[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mihiarc/pyfia/blob/main/notebooks/05_validation_statistics.ipynb)

---

In [ ]:
# Google Colab Setup - Run this cell first!
import sys
if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing pyFIA...")
    !pip install -q pyfia polars duckdb matplotlib rich
    
    # Download helpers.py for Colab
    import urllib.request
    helpers_url = "https://raw.githubusercontent.com/mihiarc/pyfia/main/notebooks/helpers.py"
    urllib.request.urlretrieve(helpers_url, "helpers.py")
    print("Setup complete! You may now run the remaining cells.")
else:
    print("Running locally - no additional setup needed.")

# Validation and Statistical Rigor

This notebook covers the statistical methodology behind pyFIA and how to validate results against official USFS estimates.

## What You'll Learn

1. Statistical foundation (Bechtold & Patterson 2005)
2. Understanding variance and confidence intervals
3. Introduction to EVALIDator (official USFS tool)
4. Running validation comparisons
5. Interpreting differences
6. Best practices and common pitfalls

**Prerequisites**: Complete Notebooks 1-4

**Estimated time**: 40 minutes

---

## Setup

In [ ]:
# Core imports
from pyfia import (
    FIA,
    area,
    volume,
    biomass,
    tpa,
    join_species_names,
)
import polars as pl
import matplotlib.pyplot as plt
import numpy as np

# Notebook helpers
from helpers import ensure_ri_data, display_estimate, plot_by_category

# Ensure data is available
db_path = ensure_ri_data()
print("Ready to begin!")

---

## 1. Statistical Foundation

### The FIA Sampling Design

FIA uses a **stratified systematic sampling design**:

1. **Systematic grid**: Plots are located on a fixed grid (approximately 1 plot per 6,000 acres)
2. **Stratification**: Plots are grouped into strata based on land cover
3. **Expansion factors**: Each plot represents a known area (stratum acres / plots in stratum)

### Bechtold & Patterson (2005)

pyFIA implements the official FIA estimation methodology described in:

> Bechtold, W.A. and Patterson, P.L. (eds). 2005. **The Enhanced Forest Inventory and Analysis Program - National Sampling Design and Estimation Procedures.** Gen. Tech. Rep. SRS-80. Asheville, NC: U.S. Department of Agriculture, Forest Service, Southern Research Station.

Key aspects:

- **Design-based inference**: Randomness comes from sample selection, not population model
- **Ratio-of-means estimator**: Estimates are ratios (e.g., volume/area)
- **Stratified estimation**: Variance accounts for stratification

### The Ratio-of-Means Estimator

For a ratio estimate like "cubic feet per acre":

$$\hat{R} = \frac{\sum_{h} \sum_{i} w_{hi} y_{hi}}{\sum_{h} \sum_{i} w_{hi} x_{hi}}$$

Where:
- $h$ = stratum
- $i$ = plot within stratum
- $w_{hi}$ = expansion factor for plot $i$ in stratum $h$
- $y_{hi}$ = response variable (e.g., volume)
- $x_{hi}$ = size variable (e.g., forest area proportion)

---

## 2. Understanding Variance and Uncertainty

### Why Uncertainty Matters

FIA estimates are based on **samples**, not complete censuses. Every estimate has uncertainty.

**Always report uncertainty** when using FIA data. An estimate without standard error is incomplete.

In [ ]:
with FIA(db_path) as db:
    db.clip_most_recent()
    result = area(db)

estimate = result["AREA"][0]
se = result["AREA_SE"][0]
se_pct = result["AREA_SE_PERCENT"][0]

print("Forest Area Estimate:")
print(f"  Point estimate: {estimate:,.0f} acres")
print(f"  Standard error: ±{se:,.0f} acres ({se_pct:.1f}%)")

### Confidence Intervals

The **95% confidence interval** is approximately:

$$\text{Estimate} \pm 1.96 \times \text{SE}$$

This means: If we repeated the sampling many times, ~95% of the intervals would contain the true value.

In [ ]:
# Calculate confidence intervals
z_95 = 1.96  # For 95% CI
z_90 = 1.645  # For 90% CI

ci_95_lower = estimate - z_95 * se
ci_95_upper = estimate + z_95 * se

ci_90_lower = estimate - z_90 * se
ci_90_upper = estimate + z_90 * se

print(f"95% Confidence Interval: {ci_95_lower:,.0f} to {ci_95_upper:,.0f} acres")
print(f"90% Confidence Interval: {ci_90_lower:,.0f} to {ci_90_upper:,.0f} acres")

In [ ]:
# Visualize the confidence interval
fig, ax = plt.subplots(figsize=(10, 4))

# Plot the normal distribution
x = np.linspace(estimate - 4*se, estimate + 4*se, 1000)
y = np.exp(-0.5 * ((x - estimate) / se) ** 2) / (se * np.sqrt(2 * np.pi))

ax.plot(x, y, color='#1565C0', linewidth=2)
ax.fill_between(x, y, where=(x >= ci_95_lower) & (x <= ci_95_upper), 
                alpha=0.3, color='#1565C0', label='95% CI')
ax.axvline(estimate, color='#C62828', linestyle='--', linewidth=2, label='Point estimate')

ax.set_xlabel('Forest Area (acres)')
ax.set_ylabel('Probability Density')
ax.set_title('Sampling Distribution of Forest Area Estimate', fontweight='bold')
ax.legend()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Format x-axis with thousands separator
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.tight_layout()
plt.show()

### Standard Error vs. Variance

| Metric | Formula | Use Case |
|--------|---------|----------|
| Variance | $\sigma^2$ | Combining estimates, pooling |
| Standard Error | $\sqrt{\sigma^2}$ | Reporting uncertainty, CI |
| CV (%) | $(SE / Estimate) \times 100$ | Comparing precision across estimates |

In [ ]:
with FIA(db_path) as db:
    db.clip_most_recent()
    
    # With standard error (default)
    result_se = area(db)
    
    # With variance
    result_var = area(db, variance=True)

print("Standard Error approach:")
print(f"  SE: {result_se['AREA_SE'][0]:,.0f} acres")
print(f"  SE%: {result_se['AREA_SE_PERCENT'][0]:.2f}%")

print(f"\nVariance approach:")
print(f"  Variance: {result_var['AREA_VARIANCE'][0]:,.0f} acres²")
print(f"  SE = sqrt(Variance): {np.sqrt(result_var['AREA_VARIANCE'][0]):,.0f} acres")

---

## 3. Precision Guidelines

### Interpreting Standard Error Percentage

| SE% | Interpretation | Recommendation |
|-----|----------------|----------------|
| < 5% | Excellent precision | Highly reliable |
| 5-10% | Good precision | Reliable for most uses |
| 10-20% | Moderate precision | Use with caution |
| 20-50% | Poor precision | Consider aggregating |
| > 50% | Very poor precision | May not be meaningful |

In [ ]:
# Compare precision across different queries
with FIA(db_path) as db:
    db.clip_most_recent()
    
    # Total state - good precision
    total = area(db)
    
    # By ownership - moderate precision  
    by_ownership = area(db, grp_by="OWNGRPCD")
    
    # By forest type - varies widely
    by_fortype = area(db, grp_by="FORTYPCD")

print("Precision Comparison:")
print(f"\nTotal State: SE% = {total['AREA_SE_PERCENT'][0]:.1f}%")

print(f"\nBy Ownership:")
for row in by_ownership.iter_rows(named=True):
    print(f"  OWNGRPCD {row['OWNGRPCD']}: SE% = {row['AREA_SE_PERCENT']:.1f}%")

print(f"\nBy Forest Type (showing high SE% examples):")
high_se = by_fortype.filter(pl.col("AREA_SE_PERCENT") > 30).sort("AREA_SE_PERCENT", descending=True).head(5)
for row in high_se.iter_rows(named=True):
    print(f"  FORTYPCD {row['FORTYPCD']}: SE% = {row['AREA_SE_PERCENT']:.1f}% (N_PLOTS={row['N_PLOTS']})")

### Why Small Categories Have Poor Precision

Precision depends on:
1. **Sample size** (number of plots)
2. **Variability** within the population
3. **How rare** the category is

For rare categories (few plots), standard errors are large. **Aggregate** when needed.

In [ ]:
# Demonstrate: Aggregating improves precision
with FIA(db_path) as db:
    db.clip_most_recent()
    
    # Individual minor forest types - poor precision
    minor_types = area(db, area_domain="FORTYPCD >= 800 AND FORTYPCD < 900")
    
    # Compare to detailed breakdown
    minor_detail = area(db, grp_by="FORTYPCD", 
                        area_domain="FORTYPCD >= 800 AND FORTYPCD < 900")

print("Effect of Aggregation on Precision:")
print(f"\nAggregated (all types 800-899):")
print(f"  Area: {minor_types['AREA'][0]:,.0f} acres")
print(f"  SE%: {minor_types['AREA_SE_PERCENT'][0]:.1f}%")
print(f"  N_plots: {minor_types['N_PLOTS'][0]}")

print(f"\nDetailed (by individual type):")
for row in minor_detail.sort("AREA", descending=True).head(3).iter_rows(named=True):
    print(f"  Type {row['FORTYPCD']}: {row['AREA']:,.0f} acres, SE%: {row['AREA_SE_PERCENT']:.1f}%, N={row['N_PLOTS']}")

---

## 4. Introduction to EVALIDator

### What is EVALIDator?

**EVALIDator** is the official USFS tool for accessing FIA estimates:
- Web interface: https://apps.fs.usda.gov/fiadb-api/evalidator
- Provides pre-computed estimates with proper variance
- The **reference standard** for FIA estimates

### Why Validate Against EVALIDator?

pyFIA implements the same methodology as EVALIDator. Validation ensures:
1. Our calculations are correct
2. We're using the same data (EVALIDs)
3. Users can trust pyFIA results

### Expected Differences

Small differences (< 1%) may occur due to:
- Rounding in intermediate calculations
- Different handling of edge cases
- Database version differences

Large differences usually indicate:
- Different EVALIDs selected
- Different filters applied
- Bug in the code

---

## 5. Running Validation Comparisons

Let's validate some pyFIA estimates by comparing them to expected EVALIDator values.

In [ ]:
def compare_to_reference(pyfia_value, reference_value, metric_name, tolerance_pct=1.0):
    """
    Compare a pyFIA estimate to a reference value.
    
    Parameters
    ----------
    pyfia_value : float
        Value from pyFIA
    reference_value : float
        Expected/reference value (e.g., from EVALIDator)
    metric_name : str
        Name of the metric being compared
    tolerance_pct : float
        Acceptable difference percentage (default 1.0%)
    """
    diff = pyfia_value - reference_value
    diff_pct = (diff / reference_value) * 100 if reference_value != 0 else 0
    
    status = "PASS" if abs(diff_pct) <= tolerance_pct else "CHECK"
    
    print(f"\n{metric_name}:")
    print(f"  pyFIA:     {pyfia_value:>15,.0f}")
    print(f"  Reference: {reference_value:>15,.0f}")
    print(f"  Diff:      {diff:>+15,.0f} ({diff_pct:+.2f}%) [{status}]")

In [ ]:
# Run pyFIA estimates
with FIA(db_path) as db:
    db.clip_most_recent()
    
    # Get the EVALID being used
    evalid = db.evalid
    print(f"Using EVALID: {evalid}")
    
    # Forest area
    area_result = area(db)
    
    # Total volume
    vol_result = volume(db)

### Manual Validation Process

To validate against EVALIDator:

1. Go to https://apps.fs.usda.gov/fiadb-api/evalidator
2. Select the same state and EVALID as pyFIA
3. Request the same metric (e.g., "Forest area")
4. Compare the values

**Note**: EVALIDator results depend on the specific report selected. Make sure to match:
- Evaluation ID
- Land basis (forest land vs timberland)
- Tree filters (if any)

In [ ]:
# Example validation output (replace reference values with actual EVALIDator results)
# These are placeholder values - actual validation requires checking EVALIDator

print("="*60)
print("VALIDATION RESULTS (Example - verify with actual EVALIDator)")
print("="*60)
print(f"\nEVALID: {evalid if 'evalid' in dir() else 'N/A'}")

# Display pyFIA results
if 'area_result' in dir():
    print(f"\nForest Area:")
    print(f"  pyFIA estimate: {area_result['AREA'][0]:,.0f} acres")
    print(f"  pyFIA SE:       {area_result['AREA_SE'][0]:,.0f} acres")
    print(f"  (Compare to EVALIDator for validation)")

if 'vol_result' in dir():
    print(f"\nNet Volume:")
    print(f"  pyFIA estimate: {vol_result['VOLCFNET_TOTAL'][0]:,.0f} cubic feet")
    print(f"  pyFIA SE:       {vol_result['VOLCFNET_TOTAL'][0] * vol_result['VOLCFNET_ACRE_SE'][0] / vol_result['VOLCFNET_ACRE'][0]:,.0f} cubic feet")
    print(f"  (Compare to EVALIDator for validation)")

---

## 6. Common Validation Issues

### Issue 1: Different EVALIDs

The most common cause of large differences. Always check which EVALID is selected.

In [ ]:
with FIA(db_path) as db:
    # Check available EVALIDs
    all_evalids = db.find_evalid()
    print("Available EVALIDs:")
    for eid in sorted(all_evalids):
        print(f"  {eid}")
    
    # Show which one is selected
    db.clip_most_recent()
    print(f"\nSelected by clip_most_recent(): {db.evalid}")

### Issue 2: Land Basis Mismatch

Make sure you're comparing the same land basis.

In [ ]:
with FIA(db_path) as db:
    db.clip_most_recent()
    
    forest = area(db, land_type="forest")
    timber = area(db, land_type="timber")

print("Land Basis Comparison:")
print(f"  Forest land: {forest['AREA'][0]:,.0f} acres")
print(f"  Timberland:  {timber['AREA'][0]:,.0f} acres")
print(f"\nDifference: {forest['AREA'][0] - timber['AREA'][0]:,.0f} acres")
print("\nNote: EVALIDator reports can be for forest land OR timberland.")
print("Make sure to match the land basis when comparing!")

### Issue 3: Filter Differences

Domain filters must match exactly.

In [ ]:
with FIA(db_path) as db:
    db.clip_most_recent()
    
    # These are different queries!
    all_live = volume(db, tree_type="live")
    gs_only = volume(db, tree_type="gs")  # Growing stock

print("Tree Type Filter Comparison:")
print(f"  All live trees:    {all_live['VOLCFNET_TOTAL'][0]/1e6:,.1f} million cuft")
print(f"  Growing stock:     {gs_only['VOLCFNET_TOTAL'][0]/1e6:,.1f} million cuft")
print(f"\nDifference: {(all_live['VOLCFNET_TOTAL'][0] - gs_only['VOLCFNET_TOTAL'][0])/1e6:,.1f} million cuft")

---

## 7. Best Practices

### DO:

1. **Always report uncertainty** - Include SE or CI with estimates
2. **Check EVALID** - Verify you're using the intended evaluation
3. **Aggregate when needed** - Combine rare categories to improve precision
4. **Document your queries** - Record the exact filters used
5. **Validate important results** - Cross-check against EVALIDator

### DON'T:

1. **Report estimates with SE% > 50%** without strong caveats
2. **Compare across EVALIDs** without accounting for temporal differences
3. **Ignore NULL values** - They may represent meaningful categories
4. **Over-stratify** - Too many groups = poor precision
5. **Double-count** - Be careful with overlapping evaluations

In [ ]:
# Example: Proper reporting with uncertainty
with FIA(db_path) as db:
    db.clip_most_recent()
    result = volume(db, by_species=True)
    
    # Add species names (pass db for reference table lookup)
    result_named = join_species_names(result, db)

top = result_named.sort("VOLCFNET_TOTAL", descending=True).head(5)

print("Example: Proper Reporting with Uncertainty")
print("="*70)
print(f"\n{'Species':<30} {'Volume (M cuft)':>15} {'95% CI':>20}")
print("-"*70)

for row in top.iter_rows(named=True):
    vol = row['VOLCFNET_TOTAL'] / 1e6
    se = row['VOLCFNET_ACRE_SE'] / row['VOLCFNET_ACRE'] * row['VOLCFNET_TOTAL'] / 1e6 if row['VOLCFNET_ACRE'] > 0 else 0
    ci_lower = vol - 1.96 * se
    ci_upper = vol + 1.96 * se
    
    print(f"{row['SPCD_NAME']:<30} {vol:>15.1f} ({ci_lower:.1f} - {ci_upper:.1f})")

print("\nNote: Always include confidence intervals for important estimates.")

---

## 8. Common Pitfalls

### Pitfall 1: Ignoring Low Sample Sizes

In [ ]:
with FIA(db_path) as db:
    db.clip_most_recent()
    result = area(db, grp_by="FORTYPCD")

# Find estimates based on very few plots
low_sample = result.filter(pl.col("N_PLOTS") <= 3)

print("Estimates with N_PLOTS <= 3:")
print(f"  Count: {len(low_sample)} forest types")
print(f"\nExample entries:")
for row in low_sample.head(5).iter_rows(named=True):
    print(f"  Type {row['FORTYPCD']}: {row['AREA']:,.0f} acres (SE% = {row['AREA_SE_PERCENT']:.0f}%, N={row['N_PLOTS']})")

print("\nThese estimates should be used with extreme caution!")

### Pitfall 2: Comparing Incompatible Estimates

In [ ]:
print("Pitfall: Comparing Different Measurements")
print("="*50)

print("\nWrong: 'The forest has 100M cuft of volume and 50M tons of biomass'")
print("       (These are different units - can't directly compare)")

print("\nWrong: 'Volume increased from 90M to 100M between 2010 and 2020'")
print("       (Without accounting for uncertainty, this may not be significant)")

# Demonstrate the uncertainty overlap
estimate_2010 = 90_000_000
se_2010 = 5_000_000
estimate_2020 = 100_000_000
se_2020 = 5_000_000

print(f"\nExample:")
print(f"  2010: {estimate_2010/1e6:.0f}M ± {1.96*se_2010/1e6:.0f}M (95% CI: {(estimate_2010-1.96*se_2010)/1e6:.0f}M - {(estimate_2010+1.96*se_2010)/1e6:.0f}M)")
print(f"  2020: {estimate_2020/1e6:.0f}M ± {1.96*se_2020/1e6:.0f}M (95% CI: {(estimate_2020-1.96*se_2020)/1e6:.0f}M - {(estimate_2020+1.96*se_2020)/1e6:.0f}M)")
print(f"\n  Note: CIs overlap ({(estimate_2010+1.96*se_2010)/1e6:.0f}M > {(estimate_2020-1.96*se_2020)/1e6:.0f}M), so change may not be significant!")

---

## Exercise 1: Precision Analysis

**Task**: Compare precision of volume estimates at different geographic scales.

1. Calculate volume for the whole state
2. Calculate volume for each ownership category
3. Calculate volume for each forest type
4. Compare the SE% at each level
5. How many forest types have SE% > 25%?

In [ ]:
# Your code here


<details>
<summary><b>Click to reveal solution</b></summary>

```python
with FIA(db_path) as db:
    db.clip_most_recent()
    
    vol_state = volume(db)
    vol_own = volume(db, grp_by="OWNGRPCD")
    vol_fort = volume(db, grp_by="FORTYPCD")

# Calculate SE% for state
state_se_pct = vol_state['VOLCFNET_ACRE_SE'][0] / vol_state['VOLCFNET_ACRE'][0] * 100

print("Precision Comparison by Geographic Scale:")
print("="*50)
print(f"\nState Total: SE% = {state_se_pct:.1f}%")

print(f"\nBy Ownership:")
for row in vol_own.iter_rows(named=True):
    se_pct = row['VOLCFNET_ACRE_SE'] / row['VOLCFNET_ACRE'] * 100 if row['VOLCFNET_ACRE'] > 0 else 0
    print(f"  OWNGRPCD {row['OWNGRPCD']}: SE% = {se_pct:.1f}%")

# Forest type analysis
vol_fort_with_se = vol_fort.with_columns(
    (pl.col("VOLCFNET_ACRE_SE") / pl.col("VOLCFNET_ACRE") * 100).alias("SE_PCT")
)

high_se = vol_fort_with_se.filter(pl.col("SE_PCT") > 25)

print(f"\nForest Type Summary:")
print(f"  Total forest types: {len(vol_fort)}")
print(f"  With SE% > 25%: {len(high_se)} ({len(high_se)/len(vol_fort)*100:.0f}%)")
print(f"  Mean SE%: {vol_fort_with_se['SE_PCT'].mean():.1f}%")
print(f"  Median SE%: {vol_fort_with_se['SE_PCT'].median():.1f}%")
```

</details>

---

## Exercise 2: Significant Differences

**Task**: Determine if the volume per acre on private land is significantly different from public land.

1. Calculate volume/acre for private ownership (OWNGRPCD=40)
2. Calculate volume/acre for public ownership (OWNGRPCD IN 10,20,30)
3. Calculate the 95% confidence intervals for each
4. Determine if the CIs overlap (if not, the difference is statistically significant)

**Hint**: The difference is significant at 95% confidence if:
- Private CI lower bound > Public CI upper bound, OR
- Public CI lower bound > Private CI upper bound

In [ ]:
# Your code here


<details>
<summary><b>Click to reveal solution</b></summary>

```python
with FIA(db_path) as db:
    db.clip_most_recent()
    
    private = volume(db, area_domain="OWNGRPCD == 40")
    public = volume(db, area_domain="OWNGRPCD IN (10, 20, 30)")

# Extract values
priv_est = private['VOLCFNET_ACRE'][0]
priv_se = private['VOLCFNET_ACRE_SE'][0]
priv_ci_low = priv_est - 1.96 * priv_se
priv_ci_high = priv_est + 1.96 * priv_se

pub_est = public['VOLCFNET_ACRE'][0]
pub_se = public['VOLCFNET_ACRE_SE'][0]
pub_ci_low = pub_est - 1.96 * pub_se
pub_ci_high = pub_est + 1.96 * pub_se

print("Volume per Acre Comparison:")
print("="*60)
print(f"\nPrivate Land:")
print(f"  Estimate: {priv_est:,.0f} cuft/acre")
print(f"  95% CI:   {priv_ci_low:,.0f} to {priv_ci_high:,.0f}")

print(f"\nPublic Land:")
print(f"  Estimate: {pub_est:,.0f} cuft/acre")
print(f"  95% CI:   {pub_ci_low:,.0f} to {pub_ci_high:,.0f}")

# Test for overlap
if priv_ci_low > pub_ci_high:
    print(f"\nConclusion: Private land has SIGNIFICANTLY HIGHER volume/acre")
    print(f"  (Private CI lower bound {priv_ci_low:,.0f} > Public CI upper bound {pub_ci_high:,.0f})")
elif pub_ci_low > priv_ci_high:
    print(f"\nConclusion: Public land has SIGNIFICANTLY HIGHER volume/acre")
    print(f"  (Public CI lower bound {pub_ci_low:,.0f} > Private CI upper bound {priv_ci_high:,.0f})")
else:
    print(f"\nConclusion: NO SIGNIFICANT DIFFERENCE at 95% confidence")
    print(f"  (Confidence intervals overlap)")
```

</details>

---

## Summary

In this notebook, you learned:

1. **Statistical foundation** - FIA uses stratified systematic sampling with ratio-of-means estimation (Bechtold & Patterson 2005)

2. **Variance and confidence intervals**
   - Standard error quantifies uncertainty
   - 95% CI ≈ Estimate ± 1.96 × SE
   - SE% indicates precision quality

3. **EVALIDator validation**
   - Official USFS tool for reference estimates
   - Use same EVALID and filters for comparison
   - Small differences (< 1%) are expected

4. **Common issues**
   - Different EVALIDs
   - Land basis mismatch (forest vs timber)
   - Filter differences

5. **Best practices**
   - Always report uncertainty
   - Aggregate rare categories
   - Validate important results
   - Document your queries

### Key Takeaways

- **No estimate is complete without uncertainty**
- **Precision depends on sample size and variability**
- **Validate against EVALIDator for important analyses**
- **Use confidence intervals to assess significance**

## Congratulations!

You've completed the pyFIA tutorial series! You now have the skills to:

- Download and access FIA data
- Estimate forest area, volume, biomass, and TPA
- Apply domain filters for custom analyses
- Analyze forest change (growth, mortality, removals)
- Understand and report statistical uncertainty

## Next Steps

- **Apply pyFIA** to your own research questions
- **Read the documentation** at docs/
- **Explore the source code** at src/pyfia/
- **Consult Bechtold & Patterson (2005)** for statistical details